In [51]:
import pandas as pd
from pandas import Series
from pandas import DataFrame
import numpy as np

pandas基于numpy的数组结构创建的, 虽然保留了numpy的数组操作, 但其主要任务为处理表格和混杂数据

### 1.1 Series
#### 1. Series结构 = index + 一维数组.  
  1. index获取 : Series.index属性  
    1. 创建Series时, 若没有指定index, 则默认添加从0开始递增1的索引
    2. 若想创建Series时指定索引, 可以使用Series(values,index=[..])
  2. 一维数组获取 : Series.values属性  


In [18]:
obj = Series([1,3,2,5])
# Series显示 : 左侧为index,右侧为一维数组
print obj  
print "=============[1]============"
print obj.index
print obj.values
print "=============[2]============"
# 创建带index的Series
obj = Series([1,2,3,4],index=['a','b','c','d'])
obj.index

0    1
1    3
2    2
3    5
dtype: int64
=============[1]============
RangeIndex(start=0, stop=4, step=1)
[1 3 2 5]
=============[2]============


Index([u'a', u'b', u'c', u'd'], dtype='object')

#### 2. 与numpy不同, pandas可以通过索引选取元素  
  1. 使用Series[index]选取元素
  2. 使用Series[index] = newVal更新元素值
  3. 使用Series[[index1,index2..]]选取多个元素, 返回一个新的Series(index不变)
  3. 使用numpy的数组函数会保留Series的index. 仍返回Series

In [27]:
print "obj[a] = %s" % obj['a']
print "=============[1]============"
obj2 = obj[['a','b']]
obj2['b'] = 99
print obj2
print "========numpy的数组函数不改变Series的index======="
np.exp(obj2)

obj[a] = 1
=============[1]============
a     1
b    99
dtype: int64
========numpy的数组函数不改变Series的index=======


a    2.718282e+00
b    9.889030e+42
dtype: float64

#### 3. Series与字典
1. 可以看做一个定长有序字典, 因为他是多对(index->value)的映射. 因此Series可以使用大部分字典函数
2. 可以通过字典直接创建Series. 字典为(rowIdx->value)
3. 仅通过字典创建的Series, 其元素自动按照字典的key进行排列.若创建Series时指定index的顺序, 则Series元素按照index的顺序排序
4. 可以使用pandas的isnull判断Series的字典中的value是否为NaN数据 (空数据)

In [45]:
# 使用字典函数操作Series
print ('a' in obj)
# 通过字典直接创建Series
data = {'d':9,'b':5,'c':7,'a':1}
obj = Series(data)  # 自动排列
print data
print obj
print "=============[1]============"
obj = Series(data,index=['b','a','c','d'])
print obj
print "=============[2]============"
print pd.isnull(obj) # value是否为NaN

True
{'a': 1, 'c': 7, 'b': 5, 'd': 9}
a    1
b    5
c    7
d    9
dtype: int64
=============[1]============
b    5
a    1
c    7
d    9
dtype: int64
=============[2]============
b    False
a    False
c    False
d    False
dtype: bool


#### 4. Series的一些其他概念
1. Series之间进行数据运算时, 会按照index对齐数据
2. Series自身和其index对象, 都有name属性, 该属性和其他关键功能密切相关
3. Series的index可以通过赋值而改变

In [50]:
# index对齐数据
obj1 = Series([1,1,1,1],index=["a","b","c","d"])
obj2 = Series([1,2,3,4],index=["a","b","c","e"])
print obj1+obj2
print "=============[1]============"
# name属性
obj1.name = "population"
obj1.index.name = "state"
print obj1
print "=============[2]============"
# index改变
obj1.index = ['aa','bb','cc','dd']
print obj1

a    2.0
b    3.0
c    4.0
d    NaN
e    NaN
dtype: float64
=============[1]============
state
a    1
b    1
c    1
d    1
Name: population, dtype: int64
=============[2]============
aa    1
bb    1
cc    1
dd    1
Name: population, dtype: int64


## 1.2 DataFrame

#### 1. DataFrame基本概念
1. 一个DataFrame就是一个表格.由一组Series组成. 这些Series长度相同, 且共用一个索引
2. DataFrame也可看做Series组成的字典,字典形式为(columnIdx->Series)
3. DataFrame有航索引和列索引

#### 2. DataFrame创建
1. DataFrame也可以看做一个字典, 其字典形式为(columnIdx,一维数组).因此可以通过字典创建DataFrame, 
2. 若创建时同时指定了字典和columnIdx, 则会按照columnIdx的顺序构造DataFrame.   
 若columnIdx中包含字典中不存在的key, 则DataFrame这列值为NaN

In [56]:
data = {'name':['zhangsan','lisi','wangwu'],
       'age':[22,27,25]}
df = DataFrame(data)
print df
print ("=============[1]============")
df2 = DataFrame(data,columns=['age','name','score'])
print df2

   age      name
0   22  zhangsan
1   27      lisi
2   25    wangwu
=============[1]============
   age      name score
0   22  zhangsan   NaN
1   27      lisi   NaN
2   25    wangwu   NaN


#### 3. DataFrame元素选取
1. 通过DataFrame对应字典的key,选取一维数组. 返回Series
2. DataFrame对应字典的key已经作为DataFrame的属性存在. 因此可以通过DataFrame."key name"选取一维数组Series
[注]: 无论哪种方式选取DataFrame中的Series, 返回的Series都和DataFrame有相同的索引(但不共用)

In [62]:
print df['age']
print ("=============[1]============")
obj = df.age
print obj

a    22
b    27
c    25
Name: age, dtype: int64
=============[1]============
a    22
b    27
c    25
Name: age, dtype: int64


#### 4. 通过赋值方式更新DataFrame的Series
1. 指定DataFrame的某列Series为一个标量, 则该Series的所有元素值都等于这个标量值
2. 指定DataFrame的某列Series为一个新的Series, 则这两个Series之间的rowindex会一一对应进行匹配
3. del关键字可以物理删除DataFrame的一个Series
4. 通过DataFrame['newColumnIdx']添加一个新的Series

In [68]:
df2['score'] = 100
print df2
print ("==============[1]============")
df2['score'] = Series([99,100,98])
print df2
print ("==============[2]============")
del df2['score']
print df2
print ("==============[3]============")
df2['ageBool'] = df2['age']>25
print df2

   age      name  score
0   22  zhangsan    100
1   27      lisi    100
2   25    wangwu    100
==============[1]============
   age      name  score
0   22  zhangsan     99
1   27      lisi    100
2   25    wangwu     98
==============[2]============
   age      name
0   22  zhangsan
1   27      lisi
2   25    wangwu
==============[3]============
   age      name  ageBool
0   22  zhangsan    False
1   27      lisi     True
2   25    wangwu    False


#### 5. DataFrame的其他概念
1. DataFrame的创建方式有很多种, 例如
  1. DataFrame(二维ndArray)
  2. DataFrame(dict('columnIdx'->List))
  3. DataFrame(dict('columnIdx'->Series))
  4. DataFrame(dict('columnIdx'->dict('rowIdx'->标量值) ))
2. DataFrame的index和columns都由name属性